# Option 1 & 2: example input

In [46]:
import pandas as pd
import pm4py

from pm4py.objects.dcr.hierarchical.obj import HierarchicalDcrGraph
graph = HierarchicalDcrGraph()

events = [
    "A", "B", "C", "D", "E", "F"
]

groups = [    
    "N1", "N2", "N3"
]

for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)


for event in groups:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    #graph.nestedgroups_map[event] = event

graph.nestedgroups["N1"] =  {"B", "N2"} 
graph.nestedgroups["N2"] =  {"C", "D"} 
graph.nestedgroups["N3"] =  {"E", "F"} 

for group, events in graph.nestedgroups.items():
    for e in events:
        graph.nestedgroups_map[e] = group

graph.marking.included.remove("B")
graph.marking.pending.add("D")


graph.conditions["N1"] = {"A"}
graph.conditions["N3"] = {"N1"}

graph.responses["N2"] = {"N3"}

graph.includes["N3"] = {"B"}

graph.excludes["C"] = {"N2"}



In [47]:
import pandas as pd
from io import StringIO
import pm4py
from pm4py.objects.dcr.hierarchical.semantics import HierarchicalSemantics  # Should be HierarchicalDcrSemantics instead!

semantics = HierarchicalSemantics()

errors = 0

expected_enabled = ["A"]
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")        


semantics.execute(graph, "A")
expected_enabled = ["A", "C", "D"]
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")                

semantics.execute(graph, "C")
expected_enabled = ["A", "E", "F"]
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")                

semantics.execute(graph, "E")
expected_enabled = ["A", "B"]
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")          

semantics.execute(graph, "B")
expected_enabled = ["A", "B", "E", "F"]
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should NOT be accepting")                

semantics.execute(graph, "F")
expected_enabled = ["A", "B", "E", "F"]
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 
                raise Exception("Event should NOT be enabled")
if not semantics.is_accepting(graph):
        errors+=1 
        raise Exception("Graph should be accepting")                        



errors # This should be 0



0